# Playground for ASFPy module development

In [1]:
import os
import csv
import random
import operator
from copy import deepcopy

from asfpy import asfpy

In [2]:
from pathlib import Path

FOLDER = "asfp2020"
EDITORS_FILE = "processed-editors-list-draft.csv"
APPLICANTS_FILE = "processed-applicants-list-draft.csv"
MATCHINGS_FILE = "matchings-list-draft.csv"
UNMATCHED_FILE = "unmatched-list-draft.csv"
MANIFEST_FILE = "applicant-id-manifest-draft.csv"

path = Path("__file__").parent.absolute()

editors_filename = path.parent.joinpath(FOLDER).joinpath(EDITORS_FILE)
editors = asfpy.read_preprocessed_editors_list_csv(editors_filename)

applicants_filename = path.parent.joinpath(FOLDER).joinpath(APPLICANTS_FILE)
applicants = asfpy.read_preprocessed_applicants_list_csv(applicants_filename)

matchings_filename = path.parent.joinpath(FOLDER).joinpath(MATCHINGS_FILE)
unmatched_filename = path.parent.joinpath(FOLDER).joinpath(UNMATCHED_FILE)
applicant_id_manifest_filename = path.parent.joinpath(FOLDER).joinpath(MANIFEST_FILE)

In [3]:
_editors = deepcopy(editors)
_applicants = deepcopy(asfpy.randomize(applicants))

p_applicants = asfpy.prioritize(_applicants)

allocation_output = asfpy.allocate(p_applicants, _editors)

# Allocation Output Handling

The output from running the allocation method includes a list of matchings, a list of unmatched applicants by identifier strings, and a list of editors _after_ decrementing capacities.

## Handle saving matchings

In [4]:
dyads = asfpy.format_matchings(allocation_output["matchings"], applicants, editors)
unmatched_applicants = asfpy.compile_unmatched(allocation_output["unmatched"], p_applicants)

asfpy.write_list_to_csv(dyads, matchings_filename)
asfpy.write_list_to_csv(asfpy.format_unmatched(unmatched_applicants), unmatched_filename)
asfpy.write_list_to_csv(asfpy.format_applicant_id_manifest(p_applicants), applicant_id_manifest_filename)

print("Number of matchings: ", len(allocation_output["matchings"]))
print("Number of unmatched: ", len(allocation_output["unmatched"]))
print("Capacity left by editors: ", asfpy.capacity(allocation_output["editors"]))

Number of matchings:  153
Number of unmatched:  51
Capacity left by editors:  2


In [5]:
asfpy.capacity(asfpy.editors_by_categories(allocation_output["editors"], {'Developmental Psychology'}))

0

## New editors by capacity and unmatched applicants

This pattern should be used to re-run the allocation for a given set of editors and applicants. In particular, this would be re-run _if_ editors increase capacity. The applicants remain in the same priority ordering from the first allocation by using `p_applicants`.

In [6]:
new_capacity_editors = allocation_output["editors"]
unmatched_ids = allocation_output["unmatched"]
## Use p_applicants to keep rank order
unmatched_applicants = asfpy.compile_unmatched(unmatched_ids, p_applicants)

asfpy.allocate(unmatched_applicants, new_capacity_editors)["matchings"]

[]

In [8]:
asfpy.format_unmatched(unmatched_applicants)

[{'applicant_id': 'APP_122',
  'applicant_email': 'tharrison@uchc.edu',
  'applicant_categories': 'Clinical Psychology',
  'applicant_rank': 3},
 {'applicant_id': 'APP_198',
  'applicant_email': 'feliciag@uoregon.edu',
  'applicant_categories': 'Clinical Psychology',
  'applicant_rank': 3},
 {'applicant_id': 'APP_035',
  'applicant_email': 'garcia.yesi@gmail.com',
  'applicant_categories': 'Clinical Psychology',
  'applicant_rank': 3},
 {'applicant_id': 'APP_076',
  'applicant_email': 'mhcook@uncg.edu',
  'applicant_categories': 'Clinical Psychology',
  'applicant_rank': 3},
 {'applicant_id': 'APP_082',
  'applicant_email': 'kristenchu@ucla.edu',
  'applicant_categories': 'Clinical Psychology',
  'applicant_rank': 3},
 {'applicant_id': 'APP_158',
  'applicant_email': 'itomei@fordham.edu',
  'applicant_categories': 'Clinical Psychology',
  'applicant_rank': 3},
 {'applicant_id': 'APP_121',
  'applicant_email': 'melissamnance@gmail.com',
  'applicant_categories': 'Clinical Psychology',
 